In [1]:
%python
%pip install beautifulsoup4

UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [0]:
from pyspark.sql.types import StringType

In [3]:
import requests
from bs4 import BeautifulSoup

BASE_URL_CNPJ = "https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/"
BASE_URL_REGIME = "https://arquivos.receitafederal.gov.br/dados/cnpj/regime_tributario/"

def getlinks(baseurl):

    res = requests.get(baseurl)
    soup = BeautifulSoup(res.text, "html.parser")

    todos_links = []

    arquivos_raiz = [a["href"] for a in soup.find_all("a") if a["href"].endswith(".zip")]
    for arq in arquivos_raiz:
        todos_links.append(baseurl + arq)

    subpastas = [a["href"] for a in soup.find_all("a") if a["href"].endswith("/") and a["href"] != "../"]

    for sub in subpastas:
        url_sub = baseurl + sub
        res_sub = requests.get(url_sub)
        soup_sub = BeautifulSoup(res_sub.text, "html.parser")

        arquivos = [a["href"] for a in soup_sub.find_all("a") if a["href"].endswith(".zip")]

        for arq in arquivos:
            link_completo = url_sub + arq
            todos_links.append(link_completo)

    for link in todos_links:
        print(link)

    return todos_links


In [4]:
cnpj = getlinks(BASE_URL_CNPJ)

https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2023-05/Cnaes.zip
https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2023-05/Empresas0.zip
https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2023-05/Empresas1.zip
https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2023-05/Empresas2.zip
https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2023-05/Empresas3.zip
https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2023-05/Empresas4.zip
https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2023-05/Empresas5.zip
https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2023-05/Empresas6.zip
https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2023-05/Empresas7.zip
https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2023-05/Empresas8.zip
https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2023-05/Empresas9.zip
htt

In [5]:
regime = getlinks(BASE_URL_REGIME)

https://arquivos.receitafederal.gov.br/dados/cnpj/regime_tributario/Dicion%c3%a1rio%20de%20Dados%20dos%20Regimes%20Tribut%c3%a1rios.zip
https://arquivos.receitafederal.gov.br/dados/cnpj/regime_tributario/Imunes%20e%20Isentas.zip
https://arquivos.receitafederal.gov.br/dados/cnpj/regime_tributario/Lucro%20Arbitrado.zip
https://arquivos.receitafederal.gov.br/dados/cnpj/regime_tributario/Lucro%20Presumido.zip
https://arquivos.receitafederal.gov.br/dados/cnpj/regime_tributario/Lucro%20Real.zip


In [12]:
print(len(cnpj))
print(len(regime))

1129
5


In [13]:
#feito em maquina local

import os
import requests
from urllib.parse import urlparse

path = r"C:\Users\JOAO PC\Documents\Databricks\Discovery-Contabilizei-FIAP\database\raw"

def baixar_arquivos(lista_links, pasta_destino):
    i = 0
    for url in lista_links:
        caminho_url = urlparse(url).path.lstrip("/")  
        caminho_local = os.path.join(pasta_destino, caminho_url)

        os.makedirs(os.path.dirname(caminho_local), exist_ok=True)
        print(i)
        print(f"Baixando {url} -> {caminho_local}")
        i += 1
        res = requests.get(url, stream=True)
        with open(caminho_local, "wb") as f:
            for chunk in res.iter_content(chunk_size=8192):
                f.write(chunk)

In [14]:
baixar_arquivos(cnpj, path)

0
Baixando https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2023-05/Cnaes.zip -> C:\Users\JOAO PC\Documents\Databricks\Discovery-Contabilizei-FIAP\database\raw\dados/cnpj/dados_abertos_cnpj/2023-05/Cnaes.zip
1
Baixando https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2023-05/Empresas0.zip -> C:\Users\JOAO PC\Documents\Databricks\Discovery-Contabilizei-FIAP\database\raw\dados/cnpj/dados_abertos_cnpj/2023-05/Empresas0.zip
2
Baixando https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2023-05/Empresas1.zip -> C:\Users\JOAO PC\Documents\Databricks\Discovery-Contabilizei-FIAP\database\raw\dados/cnpj/dados_abertos_cnpj/2023-05/Empresas1.zip
3
Baixando https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2023-05/Empresas2.zip -> C:\Users\JOAO PC\Documents\Databricks\Discovery-Contabilizei-FIAP\database\raw\dados/cnpj/dados_abertos_cnpj/2023-05/Empresas2.zip
4
Baixando https://arquivos.receitafederal.gov.br/dados/cnpj/dad

In [15]:
baixar_arquivos(regime, path)

0
Baixando https://arquivos.receitafederal.gov.br/dados/cnpj/regime_tributario/Dicion%c3%a1rio%20de%20Dados%20dos%20Regimes%20Tribut%c3%a1rios.zip -> C:\Users\JOAO PC\Documents\Databricks\Discovery-Contabilizei-FIAP\database\raw\dados/cnpj/regime_tributario/Dicion%c3%a1rio%20de%20Dados%20dos%20Regimes%20Tribut%c3%a1rios.zip
1
Baixando https://arquivos.receitafederal.gov.br/dados/cnpj/regime_tributario/Imunes%20e%20Isentas.zip -> C:\Users\JOAO PC\Documents\Databricks\Discovery-Contabilizei-FIAP\database\raw\dados/cnpj/regime_tributario/Imunes%20e%20Isentas.zip
2
Baixando https://arquivos.receitafederal.gov.br/dados/cnpj/regime_tributario/Lucro%20Arbitrado.zip -> C:\Users\JOAO PC\Documents\Databricks\Discovery-Contabilizei-FIAP\database\raw\dados/cnpj/regime_tributario/Lucro%20Arbitrado.zip
3
Baixando https://arquivos.receitafederal.gov.br/dados/cnpj/regime_tributario/Lucro%20Presumido.zip -> C:\Users\JOAO PC\Documents\Databricks\Discovery-Contabilizei-FIAP\database\raw\dados/cnpj/regime

In [0]:
df_regime = spark.createDataFrame(regime, StringType())
df_cnpj = spark.createDataFrame(cnpj, StringType())

df_regime.write\
    .mode("overwrite")\
    .format("json")\
    .save("/Volumes/workspace/discovery/discovery/links/regime")

df_cnpj.write\
    .mode("overwrite")\
    .format("json")\
    .save("/Volumes/workspace/discovery/discovery/links/cnpj")